In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd
from sklearn.model_selection import train_test_split

##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [2]:
RANDOM_SEED = 17

In [3]:
dataset = pd.read_csv("data/train_data.csv",index_col=False)
rd_augmentation = pd.read_csv("data/train_rd_augmentation.csv",index_col=False)
rs_augmentation = pd.read_csv("data/train_rs_augmentation.csv",index_col=False)

In [4]:
dataset.head()

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


In [5]:
rd_augmentation.head()

,Unnamed: 0,augmented,topic_idx,title
0,0,"['인천', '→', '핀란드', '…', '휴가철', '여행객', '분통']",4,인천 → 핀란드 … 휴가철 여행객 분통
1,1,"['실리콘밸리', '겠', '다', '…', '15', '조', '원', '들여',...",4,실리콘밸리 겠 다 … 15 조 원 들여 美 전역 거점
2,2,"['이란', '외무', '긴장', '해결책', '은', '이', '경제', '전쟁'...",4,이란 외무 긴장 해결책 은 이 경제 전쟁 멈추 것
3,3,"['NYT', '클린턴', '측근', '韓', '기업', '특수', '관계', '조...",4,NYT 클린턴 측근 韓 기업 특수 관계 조명 … 사
4,4,"['트럼프', '중미', '무역', '협상', '조속', '타결', '희망']",4,트럼프 중미 무역 협상 조속 타결 희망


In [6]:
rs_augmentation.head()

,Unnamed: 0,augmented,topic_idx,title
0,0,"['인천', '항공기', '핀란드', '→', '분통', '…', '휴가철', '여...",4,인천 항공기 핀란드 → 분통 … 휴가철 여행객 결항
1,1,"['실리콘밸리', '넘어서', '겠', '다', '…', '들여', '15', '조...",4,실리콘밸리 넘어서 겠 다 … 들여 15 조 원 구글 화 전역 거점 美
2,2,"['이란', '전쟁', '이', '완화', '해결책', '은', '미국', '긴장'...",4,이란 전쟁 이 완화 해결책 은 미국 긴장 경제 외무 멈추 는 것
3,3,"['NYT', '클린턴', '측근', '韓', '기업', '조명', '관계', '맞...",4,NYT 클린턴 측근 韓 기업 조명 관계 맞물려 … 공과 사 특수 종합
4,4,"['시진핑', '트럼프', '조속', '중미', '에', '협상', '무역', '타...",4,시진핑 트럼프 조속 중미 에 협상 무역 타결 희망


In [7]:
dataset = pd.concat([dataset,rd_augmentation,rs_augmentation])

In [8]:
dataset = dataset[["title","topic_idx"]]

In [9]:
dataset

,title,topic_idx
0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
...,...,...
45649,KB 공략 미국 선진국 스티펠 과 제휴 … IB 시장 금융,1
45650,1 보 서울시 교육청 신종 코로나 확산 에 개학 연기 · 휴업 검토,2
45651,게시판 키움증권 키움 2020 투자 실전 영웅전 대회,1
45652,는 답변 하 배기동 국립 중앙 박물 관장,2


In [10]:
dataset_train, dataset_val = train_test_split(dataset,test_size = 0.2,random_state = RANDOM_SEED)

In [11]:
dataset_train.head

<bound method NDFrame.head of                                 title  topic_idx
14928          사형 네스 앰 북한 5 년 연속 티 집행          2
31856                  박치형 EBS 신임 부사장          2
31894  황총리 北 국지도발·사이버테러할수도…감시·경계 강화속보          6
5243    삼성과 적으로 만난 류중일 LG 감독 기분이 묘하네요          5
36870        원 환율 강보합 마감 연휴 앞두 변동성 제한          1
...                               ...        ...
25631     7번째 개관하는 소극장 운동 발원지 삼일로창고극장          3
34372                    토론회 정책 노조 언론          2
42297   삼강엠앤티 516억원 규모 케미컬 탱크 3척 공급계약          1
34959    제주·남부 지방에 호우특보…완도 165.5㎜ 장대비          3
19099          프로 농구 현대모비스 오리온 잡 4 연승          5

[109569 rows x 2 columns]>

In [12]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_key, label_key, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in dataset[sent_key]]
        
        if not label_key == None:
            self.mode = "train"
        else:
            self.mode = "test"
            
        if self.mode == "train":
            self.labels = [np.int32(i) for i in dataset[label_key]]
        else:
            self.labels = [np.int32(0) for i in dataset[sent_key]]

    def __getitem__(self, i):
        if self.mode == "train":
            return (self.sentences[i] + (self.labels[i], ))
        else:
            return self.sentences[i]

    def __len__(self):
        return (len(self.labels))


using cached model


In [13]:

## Setting parameters
max_len = 40
batch_size = 16
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
NUM_CLASS = 7

epochs_no_improve = 0
min_val_loss = np.Inf
n_epochs_stop = 3

data_train = BERTDataset(dataset_train, "title", "topic_idx", tok, max_len, True, False)
data_val = BERTDataset(dataset_val, "title", "topic_idx", tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=NUM_CLASS,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [14]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [15]:
num_epochs = 1

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    val_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(val_dataloader)):
        
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out,label)
        
        val_loss += loss.data.cpu().numpy()
        test_acc += calc_accuracy(out, label)
    
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    print("epoch {} val_loss {}".format(e+1, val_loss / (batch_id+1)))
        
    if val_loss < min_val_loss:
#         Save the model
        torch.save(model,"models/kobert-5-aug-3.pth")
        epochs_no_improve = 0
        min_val_loss = val_loss
    else :
        epochs_no_improve += 1

    if e > 3 and epochs_no_improve == n_epochs_stop:
        print('Early stopping!' )
        early_stop = True
        break
    else:
        print("Keep going!")
        continue
    break
    
    if early_stop:
        print("Stopped")
        break

  
        


/home/park/Workspace/dacon-new-classification/venv/lib/python3.6/site-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/6849 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9945979118347168 train acc 0.125
epoch 1 batch id 201 loss 1.473260760307312 train acc 0.40671641791044777
epoch 1 batch id 401 loss 0.3616846203804016 train acc 0.6111284289276808
epoch 1 batch id 601 loss 0.5342386960983276 train acc 0.6837562396006656
epoch 1 batch id 801 loss 0.50956791639328 train acc 0.7218320848938826
epoch 1 batch id 1001 loss 0.3578757643699646 train acc 0.7465034965034965
epoch 1 batch id 1201 loss 0.7025125622749329 train acc 0.7612926727726894
epoch 1 batch id 1401 loss 0.6026637554168701 train acc 0.7734207708779444
epoch 1 batch id 1601 loss 0.3897317349910736 train acc 0.7819331667707683
epoch 1 batch id 1801 loss 0.15961416065692902 train acc 0.7887631871182677
epoch 1 batch id 2001 loss 0.8002963066101074 train acc 0.7972263868065967
epoch 1 batch id 2201 loss 0.520969808101654 train acc 0.802277373920945
epoch 1 batch id 2401 loss 0.740515410900116 train acc 0.8072678050812162
epoch 1 batch id 2601 loss 0.27699047327041626 tr

/home/park/Workspace/dacon-new-classification/venv/lib/python3.6/site-packages/ipykernel_launcher.py:49: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1713 [00:00<?, ?it/s]

epoch 1 test acc 0.9026196730881495
epoch 1 val_loss 0.30238682804525907
Keep going!


In [18]:
dataset_test = pd.read_csv("data/test_data.csv",index_col=False)
data_test = BERTDataset(dataset_test, "title", None, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

submission = pd.read_csv('data/sample_submission.csv')

In [19]:
model = torch.load("models/kobert-5-aug-3.pth")

In [20]:
model.eval()
pred = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    _,max_indices = torch.max(out,1)
    for idx in max_indices.cpu().numpy():
        pred.append(idx)

/home/park/Workspace/dacon-new-classification/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/571 [00:00<?, ?it/s]

In [21]:
submission['topic_idx'] = pred
print(len(pred))
print(submission.head())
submission.to_csv("results/kobert-5-3.csv",index=False)

9131
   index  topic_idx
0  45654          0
1  45655          3
2  45656          2
3  45657          0
4  45658          3


/home/park/Workspace/dacon-new-classification/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/571 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.92 GiB total capacity; 8.60 GiB already allocated; 14.56 MiB free; 10.12 GiB reserved in total by PyTorch)